<a href="https://colab.research.google.com/github/AlejoGomezQ/Practica_04_Analitica_Datos/blob/main/Calidad_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perfilado de Datos - Registro Único de Vendedores de Cartagena


## 1. Importar librerías

In [5]:
!pip install ydata_profiling

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

## 2. Cargar los datos originales

In [ ]:
df = pd.read_csv('Registro__unico_de_vendedores_de_Cartagena.csv')
print(f"Dataset cargado con {df.shape[0]} filas y {df.shape[1]} columnas")
df.head()

Dataset cargado con 3416 filas y 8 columnas


,Actividad_Oferta,Localidad,Barrios,Tipo_de_Venta,Tipo_de_Oferta,Genero,Etnia,Estado
0,ARTESANIAS,'LOCALIDAD 3','ALAMEDA LA VICTORIA',ESTACIONARIO,BIENES,FEMENINO,NINGUNA,ACTIVO
1,AGUA/REFRESCOS,'LOCALIDAD 1',BAZURTO,AMBULANTE,SERVICIOS,MASCULINO,NINGUNA,ACTIVO
2,AGUA/REFRESCOS,'LOCALIDAD 1',BAZURTO,ESTACIONARIO,BIENES,FEMENINO,NINGUNA,ACTIVO
3,AGUA/REFRESCOS,'LOCALIDAD 1',BAZURTO,AMBULANTE,SERVICIOS,FEMENINO,NINGUNA,ACTIVO
4,AGUA/REFRESCOS,'LOCALIDAD 1',BAZURTO,ESTACIONARIO,BIENES,FEMENINO,NINGUNA,ACTIVO


## 3. Generar reporte de perfilado

Generamos un reporte completo de perfilado de datos que incluye:
- Estadísticas descriptivas
- Distribuciones de variables
- Correlaciones
- Valores faltantes
- Alertas de calidad de datos

In [ ]:
profile = ProfileReport(df, title="Reporte de Perfilado - Vendedores de Cartagena", explorative=True)
profile.to_file("reporte_perfilado_vendedores.html")

print("Reporte generado exitosamente: reporte_perfilado_vendedores.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

## 4. Visualizar reporte

In [ ]:
profile.to_notebook_iframe()

NameError: name 'profile' is not defined

## 5. Diagnóstico de calidad de datos

### Variables redundantes

Durante el análisis exploratorio se identificó que la variable **barrio** presenta redundancia con respecto a la variable **localidad**. Si bien las localidades mantienen nombres únicos que permiten una identificación clara, los barrios pueden compartir el mismo nombre entre diferentes localidades.

Debido a que la variable **localidad** proporciona un mayor nivel de granularidad y especificidad geográfica sin ambigüedades, se tomó la decisión de **eliminar la variable barrio** del conjunto de datos.

### Análisis de datos atípicos

Durante la revisión del conjunto de datos, se identificaron inconsistencias menores en el formato de ciertas variables categóricas:

- **Género**: Se detectaron valores registrados en minúsculas ("femenino" y "masculino").

- **Etnia**: De manera similar, se encontraron 5 registros con el valor "afrocolombiano" en minúsculas.

Estas inconsistencias fueron mínimas en cantidad y, dado que ya existían categorías correctamente definidas para estos valores, se realizó una **normalización manual** para estandarizar el formato.

Tras la corrección de estos datos atípicos, se verificó que **no existen valores nulos** en el conjunto de datos, lo que confirma la integridad y completitud del dataset.

## 6. Limpieza de Datos

### 6.1 Crear copia de seguridad del dataset original

In [ ]:
import shutil
from datetime import datetime

archivo_original = 'Registro__unico_de_vendedores_de_Cartagena.csv'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
archivo_backup = f'Registro_vendedores_backup_{timestamp}.csv'

shutil.copy2(archivo_original, archivo_backup)

### 6.2 Cargar datos y verificar estructura

In [ ]:
df_limpieza = pd.read_csv('Registro__unico_de_vendedores_de_Cartagena.csv')

print(f"Dimensiones: {df_limpieza.shape[0]} filas x {df_limpieza.shape[1]} columnas")
print(f"Valores nulos: {df_limpieza.isnull().sum().sum()}")

df_limpieza.head()

### 6.3 Eliminación de variables redundantes

In [ ]:
columnas_posibles_barrio = ['barrio', 'Barrio', 'BARRIO', 'Barrio/Vereda']

for col in columnas_posibles_barrio:
    if col in df_limpieza.columns:
        df_limpieza = df_limpieza.drop(columns=[col])
        break

df_limpieza.shape

### 6.4 Corrección de datos atípicos - Estandarización

In [ ]:
columnas_posibles_genero = ['Genero', 'genero', 'Género', 'género', 'GENERO', 'GÉNERO', 'Sexo', 'sexo']
columnas_posibles_etnia = ['Etnia', 'etnia', 'ETNIA', 'Grupo Etnico', 'grupo_etnico']

columna_genero = None
columna_etnia = None

for col in columnas_posibles_genero:
    if col in df_limpieza.columns:
        columna_genero = col
        break

for col in columnas_posibles_etnia:
    if col in df_limpieza.columns:
        columna_etnia = col
        break

In [ ]:
if columna_genero:
    df_limpieza[columna_genero] = df_limpieza[columna_genero].str.upper().str.strip()

if columna_etnia:
    df_limpieza[columna_etnia] = df_limpieza[columna_etnia].str.upper().str.strip()

df_limpieza.head()

### 6.5 Análisis de correlaciones

#### Correlaciones peligrosamente altas

Según el análisis de correlaciones del dataset, **no se evidenciaron correlaciones peligrosamente altas** entre las variables. No se hace necesario la eliminación de ninguna variable.


#### Correlaciones con la variable objetivo (ESTADO)

La variable objetivo **ESTADO** es categórica binaria con dos valores posibles: **ACTIVO** o **INACTIVO**. A continuación se presenta el análisis de correlación de las demás variables con respecto a esta variable objetivo:

**Correlaciones Normales:**

- **Etnia**: Presenta la correlación más alta con la variable objetivo (**0.2258**), aunque moderada.

- **Tipo_de_oferta**: Segunda variable con mayor correlación (**0.1348**), indicando una relación moderada-baja con el estado del vendedor.

**Correlaciones Bajas:**

- **Tipo_de_Venta**: Correlación de **0.0947**
- **Actividad_Oferta**: Correlación de **0.0873**
- **Localidad**: Correlación de **0.0584**
- **Género**: Presenta la correlación más baja (**0.0375**)

#### Interpretación y decisiones

**Variable con Mayor Correlación:**  
**Etnia** (0.2258) - Esta variable se conserva por su mayor capacidad predictiva relativa.

**Variable con Menor Correlación:**  
**Género** (0.0375) - Aunque presenta la correlación más baja.

**Conclusión**: Se mantienen todas las variables en el dataset limpio, priorizando toda la información

In [ ]:
df_correlacion = df_limpieza.copy()
columnas_categoricas = df_correlacion.select_dtypes(include=['object']).columns

le = LabelEncoder()
for col in columnas_categoricas:
    df_correlacion[col] = le.fit_transform(df_correlacion[col].astype(str))

matriz_correlacion = df_correlacion.corr()

posibles_nombres_estado = ['ESTADO', 'Estado', 'estado']
columna_estado = None

for nombre in posibles_nombres_estado:
    if nombre in df_limpieza.columns:
        columna_estado = nombre
        break

correlaciones_objetivo = matriz_correlacion[columna_estado].sort_values(ascending=False)
correlaciones_objetivo

### 6.6 Exportar datos limpios

In [ ]:
df_limpieza.to_csv('Registro_vendedores_limpio.csv', index=False)

print(f"Registros: {df_limpieza.shape[0]}, Columnas: {df_limpieza.shape[1]}")